# Homework 10

## Problem 1. 30 POINTS
Create a bash shell script that will create a sqlite database of the 4 csv files in this directory. Make sure that you assing meaningful data types and constraints to your database.

In [61]:
%%bash
cat >> bash_script.sh << EOF
#!/bin/bash
sqlite3 mydata.db
CREATE TABLE billing (
    billingid INT PRIMARY KEY,
    encounterid INT NOT NULL,
    code INT NOT NULL);

CREATE TABLE encounter (
    encounterid INT PRIMARY KEY,
    patientid INT NOT NULL,
    encounterdate DATE NOT NULL,
    encounterphysician INT NOT NULL,
    encounterreason VARCHAR(255) NOT NULL,
    billingamount FLOAT NOT NULL);

CREATE TABLE patient (
    patientid INT PRIMARY KEY,
    lastname VARCHAR(255) NOT NULL,
    firstname VARCHAR(255) NOT NULL,
    gender VARCHAR(255) NOT NULL,
    dob DATE NOT NULL);
    
CREATE TABLE physician (
    physicianid INT PRIMARY KEY,
    lastname VARCHAR(255) NOT NULL,
    firstname VARCHAR(255) NOT NULL,
    specialty VARCHAR(255) NOT NULL);

.mode csv
.import billing.csv billing
.import encounter.csv encounter
.import patient.csv patient
.import physician.csv physician
.quit
EOF

## Problem 2. 30 POINTS.
Using python, answer the following questions:
    1. Provide code that creates a pandas dataframe that contains a patient-level dataset with:
        - LastName
        - FirstName
        - Total Billing Amount
    2. The billing department needs to modify each billing statment so that we take the total billing amount for each person, multiply it by 4.2 and subtract 10. Create a new column on your pandas dataframe that applies this correction and name this variable `correction`.
    3. For all patients less than 40 years old, create a pandas data frame that contains:
        - LastName
        - FirstName
        - physician last name
        - physician first name
        - total times seen that physician
    

In [157]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from collections import OrderedDict
patient_level = create_engine('sqlite:///mydata.db')
sql1 = "SELECT patient.lastname, patient.firstname, encounter.billingamount FROM patient INNER JOIN encounter ON patient.patientid = encounter.patientid"
df0_patient_level = pd.read_sql(sql1, patient_level)
dic = df0_patient_level.values
lastname=[]
firstname=[]
total_billing=[]
for i in dic[1:]:
    if i[0] not in lastname and i[1] not in firstname:
        lastname.append(i[0])
        firstname.append(i[1])
        total_billing.append(i[2])
        if i[2] is None:
            i[2] = 0
    else:
        if i[2] is None:
            i[2] = 0
        index = lastname.index(i[0])
        total_billing[index] += i[2]
df_patient_level = pd.DataFrame(OrderedDict(lastname=lastname,
                                            firstname=firstname,
                                            total_billing=total_billing,))                                
df_patient_level

,lastname,firstname,total_billing
0,doe,john,186.97
1,sickly,joseph,888.10
2,daily,bob,375.69


In [158]:
df_patient_level['correction'] = df_patient_level['total_billing'] * 4.2 - 10

df_patient_level

,lastname,firstname,total_billing,correction
0,doe,john,186.97,775.274
1,sickly,joseph,888.10,3720.020
2,daily,bob,375.69,1567.898


In [237]:
sql2 = "SELECT patient.lastname, patient.firstname, patient.dob, physician.lastname, physician.firstname, encounter.encounterreason FROM encounter INNER JOIN patient ON patient.patientid = encounter.patientid INNER JOIN physician ON encounter.encounterphysician = physician.physicianid"
df0 = pd.read_sql(sql2, patient_level)
patient_lastname=[]
patient_firstname=[]
physician_lastname=[]
physician_firstname=[]
number_of_time_physician=[]
dic1 = df0.values
for i in dic1:
    age = 118 - int(i[2][-2:])
    if age < 40:
        if i[0] not in patient_lastname or i[1] not in patient_firstname or i[3] not in physician_lastname or i[4] not in physician_firstname:
            patient_lastname.append(i[0])
            patient_firstname.append(i[1])
            physician_lastname.append(i[3])
            physician_firstname.append(i[4])
            number_of_time_physician.append(1)
        else:
            for j in range(len(patient_lastname)):
                if i[0] == patient_lastname[j] and i[3] == physician_lastname[j]:
                    number_of_time_physician[j] += 1
sql3 = "SELECT lastname, firstname, dob FROM patient"
df1 = pd.read_sql(sql3, patient_level)
dic2 = df1.values
patient_la=[]
patient_fi=[]
phy_la=[]
phy_fir=[]
number=[]
for j in dic2[1:]:
    age = 118 - int(j[2][-2:])
    if age < 40:
        if j[0] not in patient_lastname and j[1] not in patient_firstname:
            patient_la.append(j[0])
            patient_fi.append(j[1])
            phy_la.append(None)
            phy_fir.append(None)
            number.append(None)
df2 = pd.DataFrame(OrderedDict(patient_lastname=patient_lastname,
                              patient_firstname=patient_firstname,
                              physician_lastname=physician_lastname,
                              physician_firstname=physician_firstname,
                              number_of_time_physician=number_of_time_physician,))
df3 = pd.DataFrame(OrderedDict(patient_lastname=patient_la,
                              patient_firstname=patient_fi,
                              physician_lastname=phy_la,
                              physician_firstname=phy_fir,
                              number_of_time_physician=number,))
df4=df2.append(df3, sort=False)
df4 = df4.reset_index(drop=True)
df4

,patient_lastname,patient_firstname,physician_lastname,physician_firstname,number_of_time_physician
0,sickly,joseph,wei,liu,2
1,sickly,joseph,na,li,3
2,daily,bob,na,li,1
3,daily,bob,satterwhite,jill,1
4,sickly,joseph,whitecoat,bill,1
5,well,sally,None,None,None


## Problem 1. 15 POINTS
You should have received an email that contains:
    - host
    - user
    - password
    
Go ahead and create three environment variables that hold this information. From this point forward, if you need this information, you should refer to the environment variables and NOT the values themselves. You will lose all points, if the values themselves are in the code and submitted to github. You will have effectively disclosed very sensitive data. **NOTE**, there is no code you turn in to get points for this problem. Rather, I will know that you have done this by reviewing the code below.

## Problem 2. 5 POINTS
Create a `sqlalchemy` engine that is connected to our database. **NOTE** use the environment variables set up in problem 1 to do this. There should be NO host, username, or passwords visible if you are using environment variables correctly. You may receive some `ImportError`. You will need to figure out how to resolve these issues.

In [25]:
import os
from sqlalchemy import create_engine
host = os.environ.get('host')
user = os.environ.get('user')
password = os.environ.get('password')
url = 'postgresql://' + user + ':' + password + "@" + host
engine1 = create_engine(url)

## Problem 3. 20 POINTS
Using the engine created in Problem 2. Use sqlalchemy to help you create a hand drawn ER diagram. Please find a way to get you hand-drawn ER diagram into this repository. HINT, if you take a picture with your phone, you can use that image (which is just a file) to put into this directory.

In [35]:
from sqlalchemy import inspect
inspector = inspect(engine1)
tables = inspector.get_table_names()